In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import cv2
import os
import random

# --- KONFIGURASI PATH KHUSUS KAGGLE ---
# Dataset di Kaggle biasanya ada di path ini
# Kita cek apakah foldernya double nesting atau tidak
base_path = '/kaggle/input/chest-xray-pneumonia/chest_xray'

if os.path.exists(os.path.join(base_path, 'chest_xray')):
    print("Mendeteksi struktur folder ganda...")
    base_dir = os.path.join(base_path, 'chest_xray')
else:
    base_dir = base_path

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
# Kita abaikan folder 'val' bawaan dataset karena isinya terlalu sedikit (cuma 16 gambar)
# Kita akan buat validasi sendiri dari folder train

print(f"Train Dir: {train_dir}")
print(f"Test Dir:  {test_dir}")

# Parameter
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 25 # Agak lama karena learning rate kecil (fine-tuning)

print("GPU Available:", tf.config.list_physical_devices('GPU'))

In [ ]:
# --- PART 2: DATA LOADING (EFFICIENTNET VERSION) ---
from tensorflow.keras.applications.efficientnet import preprocess_input

# Hapus rescale=1./255, ganti dengan preprocessing_function bawaan model
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, # Trik agar akurasi maksimal
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

print("Setup Generators EfficientNet...")
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

In [ ]:
# --- PART 3: MODEL ARCHITECTURE (EFFICIENTNET-B0) ---

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import backend as K

def focal_loss(gamma=2., alpha=0.25):
    def binary_focal_loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        alpha_t = y_true * alpha + (K.ones_like(y_true) - y_true) * (1 - alpha)
        p_t = y_true * y_pred + (K.ones_like(y_true) - y_true) * (1 - y_pred) + K.epsilon()
        focal_loss = - alpha_t * K.pow((K.ones_like(y_true) - p_t), gamma) * K.log(p_t)
        return K.mean(focal_loss)
    return binary_focal_loss

def build_model_efficientnet():
    # 1. Load EfficientNetB0
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
    
    # 2. FINE TUNING
    base_model.trainable = True
    
    # EfficientNetB0 punya sekitar 230+ layers
    # Kita freeze layer awal, sisakan 40 layer terakhir
    for layer in base_model.layers[:-40]:
        layer.trainable = False

    # 3. Custom Head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x) 
    predictions = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # 4. Compile (Tetap pakai LR kecil)
    model.compile(optimizer=AdamW(learning_rate=1e-5, weight_decay=1e-4),
                  loss=focal_loss(gamma=2.0, alpha=0.25),
                  metrics=['accuracy', tf.keras.metrics.Recall(name='sensitivity'), tf.keras.metrics.AUC(name='auc')])
    
    return model

model = build_model_efficientnet()
print("Model diganti menjadi: EFFICIENTNET-B0")
# model.summary() # Boleh di-uncomment kalau mau lihat strukturnya

In [ ]:
# --- PART 4: TRAINING (NEW FILENAME) ---

# Nama file dibedakan
checkpoint_path = '/kaggle/working/model_EFFICIENTNET_best.keras'
final_model_path = '/kaggle/working/model_EFFICIENTNET_final.keras'

callbacks = [
    ModelCheckpoint(checkpoint_path, monitor='val_auc', save_best_only=True, mode='max', verbose=1),
    EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7, verbose=1)
]

history = model.fit(
    train_generator,
    epochs=EPOCHS, 
    validation_data=val_generator,
    callbacks=callbacks
)

model.save(final_model_path)
print(f"Model disimpan dengan nama: {final_model_path}")

In [ ]:
# --- EVALUATION & METRICS ---

# Prediksi Data Test
predictions = model.predict(test_generator)
y_pred_binary = (predictions > 0.5).astype(int).flatten()
y_true = test_generator.classes

# Confusion Matrix
tn, fp, fn, tp = confusion_matrix(y_true, y_pred_binary).ravel()
accuracy = (tp + tn) / (tp + tn + fp + fn)
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
roc_auc = auc(*roc_curve(y_true, predictions)[:2])

print(f"\nAKURASI: {accuracy:.4f}")
print(f"SENSITIVITAS (Recall): {sensitivity:.4f}")
print(f"SPESIFITAS: {specificity:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")

# Visualisasi Grafik
plt.figure(figsize=(18, 5))

# Plot Loss & Acc
plt.subplot(1, 3, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Training Accuracy')
plt.legend()

# Confusion Matrix
plt.subplot(1, 3, 2)
sns.heatmap([[tn, fp], [fn, tp]], annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Normal', 'Pneumonia'], yticklabels=['Normal', 'Pneumonia'])
plt.title('Confusion Matrix')

# ROC Curve
fpr, tpr, _ = roc_curve(y_true, predictions)
plt.subplot(1, 3, 3)
plt.plot(fpr, tpr, color='orange', lw=2, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.title('ROC Curve')
plt.legend()

plt.show()

In [ ]:
# --- LOAD MODEL TERBAIK SECARA MANUAL ---
from tensorflow.keras.models import load_model

# Lokasi file terbaik yang baru saja kita simpan
best_model_path = '/kaggle/working/model_EFFICIENTNET_final.keras'

print(f"Sedang meload model terbaik dari: {best_model_path} ...")

# PENTING: Kita pakai compile=False karena kita cuma mau Prediksi (Predict), 
# bukan mau lanjut training. Ini menghindari error ribet soal 'focal loss'.
model = load_model(best_model_path, compile=False)

print("✅ Model BERHASIL diload! Sekarang model di memori adalah versi TERBAIK.")

In [ ]:
# --- PART 7: GRAD-CAM FUNCTION (OPTIMIZED & CLEANED) ---
import cv2
import numpy as np
import tensorflow as tf

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # 1. Buat model gradien
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # 2. Rekam operasi gradien
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # 3. Hitung gradien
    grads = tape.gradient(class_channel, last_conv_layer_output)
    
    # 4. Global Average Pooling (Rata-rata gradien)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # 5. Kalikan feature map dengan bobot gradien
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # 6. Normalisasi (ReLU & Scaling)
    heatmap = tf.maximum(heatmap, 0) / (tf.math.reduce_max(heatmap) + 1e-10) # Tambah epsilon biar gak error bagi 0
    return heatmap.numpy()

def overlay_heatmap(img_path, heatmap, alpha=0.4, threshold=0.3):
    """
    Args:
        threshold (float): Nilai 0.0 - 1.0. 
                           Area heatmap di bawah nilai ini akan dibuang (dihitamkan).
                           Berguna untuk menghilangkan noise/biru-biru di background.
    """
    # 1. Load Gambar Asli
    img = cv2.imread(img_path)
    img = cv2.resize(img, IMG_SIZE) # Pastikan ukuran sama (224, 224)
    
    # 2. Resize Heatmap agar sama persis dengan gambar asli
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    
    # --- FITUR BARU: CLEAN NOISE (THRESHOLDING) ---
    # Buang area yang panasnya kurang dari 30% (0.3)
    # Ini membuat heatmap lebih fokus, tidak melebar ke mana-mana
    heatmap[heatmap < threshold] = 0 
    # ----------------------------------------------
    
    # 3. Ubah ke format 8-bit (0-255)
    heatmap_uint8 = np.uint8(255 * heatmap)
    
    # 4. Beri Warna (JET: Biru=Dingin, Merah=Panas)
    heatmap_colored = cv2.applyColorMap(heatmap_uint8, cv2.COLORMAP_JET)
    
    # 5. Trik Visualisasi: 
    # Di colormap JET, nilai 0 (hitam) itu warnanya Biru Tua.
    # Kita tidak mau background jadi biru semua.
    # Jadi, area yang tadi kita buang (di bawah threshold), kita hitamkan kembali warnanya.
    heatmap_colored[heatmap < threshold] = 0
    
    # 6. Gabungkan (Overlay)
    superimposed_img = heatmap_colored * alpha + img
    
    # Pastikan tidak ada nilai pixel > 255
    superimposed_img = np.clip(superimposed_img, 0, 255).astype('uint8')
    
    return img, superimposed_img

In [ ]:
# --- PART 8: TEST PREDICTION (REVISED FOR EFFICIENTNET) ---
from tensorflow.keras.applications.efficientnet import preprocess_input

# Nama layer terakhir EfficientNetB0
last_conv_layer_name = "block5c_add"

# Ambil sample acak
target_class = random.choice(['PNEUMONIA', 'NORMAL']) 
sample_dir = os.path.join(test_dir, target_class)
sample_image = random.choice(os.listdir(sample_dir))
img_path = os.path.join(sample_dir, sample_image)

# --- BAGIAN PENTING (JANGAN SALAH DISINI) ---
img = tf.keras.preprocessing.image.load_img(img_path, target_size=IMG_SIZE)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

# HAPUS baris: img_array /= 255.0 (JANGAN PAKAI INI UNTUK EFFICIENTNET)
# GANTI DENGAN INI:
img_array = preprocess_input(img_array) 
# ---------------------------------------------

# Prediksi
preds = model.predict(img_array)
score = preds[0][0]

label_pred = "PNEUMONIA" if score > 0.5 else "NORMAL"
confidence = score if score > 0.5 else 1 - score

print(f"File: {sample_image}")
print(f"Ground Truth: {target_class}")
print(f"Prediksi: {label_pred} ({confidence*100:.2f}%)")

# Grad-CAM
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=0)
original_img, superimposed_img = overlay_heatmap(img_path, heatmap)

# Plot
plt.figure(figsize=(12, 6))

plt.subplot(1, 3, 1)
plt.imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
plt.title(f"Asli: {target_class}")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(heatmap, cmap='jet')
plt.title("Heatmap")
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
plt.title(f"Prediksi: {label_pred}\nConf: {confidence*100:.1f}%")
plt.axis('off')

plt.tight_layout()
plt.show()

# Validasi Text
if label_pred == target_class:
    print("✅ PREDIKSI BENAR")
else:
    print("❌ PREDIKSI SALAH")

In [ ]:
# --- PART 9 (UPGRADE): APLIKASI WEB DENGAN SLIDER ---
import gradio as gr
from tensorflow.keras.applications.efficientnet import preprocess_input

# Fungsi Prediksi menerima input tambahan: THRESHOLD
def predict_image_interactive(image, threshold_slider):
    if image is None:
        return None, None
        
    # 1. Preprocessing
    img_array = cv2.resize(image, IMG_SIZE)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    
    # 2. Prediksi
    preds = model.predict(img_array)
    score = preds[0][0]
    
    if score > 0.5:
        label = "PNEUMONIA"
        conf = score
    else:
        label = "NORMAL"
        conf = 1 - score
        
    # 3. Buat Grad-CAM
    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=0)
    
    # Resize ke ukuran asli gambar input
    heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))
    
    # --- PROSES SLIDER ---
    # Gunakan nilai dari Slider Gradio untuk membersihkan heatmap
    heatmap_clean = heatmap.copy()
    heatmap_clean[heatmap_clean < threshold_slider] = 0 
    # ---------------------
    
    # Pewarnaan
    heatmap_uint8 = np.uint8(255 * heatmap_clean)
    heatmap_colored = cv2.applyColorMap(heatmap_uint8, cv2.COLORMAP_JET)
    
    # Hapus background biru (kembalikan warna asli jika heatmap 0)
    heatmap_colored[heatmap_clean < threshold_slider] = 0
    
    # Overlay
    superimposed_img = heatmap_colored * 0.4 + image
    
    # Fix warna background yang hitam jadi gambar asli
    mask = heatmap_clean < threshold_slider
    superimposed_img[mask] = image[mask]
    
    superimposed_img = np.clip(superimposed_img, 0, 255).astype('uint8')
    
    return {label: float(conf), "Lawan": 1-float(conf)}, superimposed_img

# Setup Antarmuka dengan Slider
interface = gr.Interface(
    fn=predict_image_interactive,
    inputs=[
        gr.Image(label="1. Upload X-Ray"),
        gr.Slider(minimum=0.0, maximum=1.0, value=0.3, label="2. Sensitivitas Heatmap (Geser untuk filter noise)")
    ],
    outputs=[
        gr.Label(num_top_classes=2, label="Hasil Prediksi"),
        gr.Image(label="Visualisasi Lokasi Penyakit")
    ],
    title="🏥 Advanced Pneumonia Detector",
    description="Analisis X-Ray berbasis EfficientNet-B0 dengan Grad-CAM Interaktif.",
    # Contoh gambar biar cepat tes (ambil dari folder test)
    examples=[
        [os.path.join(test_dir, 'PNEUMONIA', os.listdir(os.path.join(test_dir, 'PNEUMONIA'))[0]), 0.3],
        [os.path.join(test_dir, 'NORMAL', os.listdir(os.path.join(test_dir, 'NORMAL'))[0]), 0.3]
    ]
)

interface.launch(share=True, debug=True)